### **FAISS**

- Facebook AI Similarity Search (Faiss)는 밀집 벡터의 효율적인 유사도 검색과 클러스터링을 위한 라이브러리

- Faiss는 RAM에 맞지 않을 수도 있는 벡터 집합을 포함하여 모든 크기의 벡터 집합을 검색하는 알고리즘을 포함

- 평가와 매개변수 튜닝을 위한 지원 코드가 있음

**참고**
- [LangChain FAISS 문서](https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/)
- [FAISS 문서](https://faiss.ai/)

In [1]:
# 데이터셋 로드
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/nlp-keywords.txt")
loader2 = TextLoader("data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

(11, 6)

### **VectorStore 생성**

기본적인 FAISS DB 생성 방법

**주요 초기화 매개변수**

1. 인덱싱 매개변수:
   - `embedding_function` (Embeddings): 사용할 임베딩 함수

2. 클라이언트 매개변수:
   - `index` (Any): 사용할 FAISS 인덱스
   - `docstore` (Docstore): 사용할 문서 저장소
   - `index_to_docstore_id` (Dict[int, str]): 인덱스에서 문서 저장소 ID로의 매핑

**참고**

- FAISS는 고성능 벡터 검색 및 클러스터링을 위한 라이브러리
- 이 클래스는 FAISS를 LangChain의 VectorStore 인터페이스와 통합
- 임베딩 함수, FAISS 인덱스, 문서 저장소를 조합하여 효율적인 벡터 검색 시스템을 구축

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 차원 크기를 계산(index에 넣음)
dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

1536


In [ ]:
# FAISS Vector Store 생성
db = FAISS(
    embedding_function=embeddings,
    index = faiss.IndexFlatL2(dimension_size),   # L2 거리를 사용하는 인덱스 
    docstore = InMemoryDocstore(),
    index_to_docstore_id = {}
)

{}

### **FAISS 벡터 저장소 생성 (from_documents)**

`from_documents` 를 이용한 Vector Store 생성

**매개변수**

- `documents` (List[Document]): 벡터 저장소에 추가할 문서 리스트
- `embedding` (Embeddings): 사용할 임베딩 함수
- `**kwargs`: 추가 키워드 인자

**동작 방식**

1. 문서 리스트에서 텍스트 내용(`page_content`)과 메타데이터
2. 추출한 텍스트와 메타데이터를 사용하여 `from_texts` 메서드 호출

In [42]:
# DB 생성
db = FAISS.from_documents(documents=split_doc1, embedding = embeddings)

# 저장소 내의 문서 ID 확인
db.index_to_docstore_id

{0: '7f617280-ecb0-472d-9ce7-f3b2b6fedb71',
 1: '716812c2-13e2-4a0a-8ae2-07a7300475b9',
 2: 'e12c13b4-b17b-48d1-ad52-2be1797770d1',
 3: 'e4ee5a0e-fb4c-411e-b597-1b2c732a3db9',
 4: '5633d481-7eb9-4268-bd78-3353089125e4',
 5: 'f23a3f29-fec4-4abb-99e6-f577f4ef2fda',
 6: '93d64887-f21e-4169-a15f-b93e2f4b07d2',
 7: '1fc5573e-e88a-4d24-861a-1ba39f80ab8b',
 8: '295b1d8e-edb7-48a1-99d4-c95c469eebff',
 9: 'aa1465c0-0a2f-4654-9aff-8e04e2f80976',
 10: 'a5d31f54-f0be-482a-b1c6-32e98f239270'}

In [43]:
# 저장된 문서의 ID: Document 확인
db.docstore._dict

{'7f617280-ecb0-472d-9ce7-f3b2b6fedb71': Document(id='7f617280-ecb0-472d-9ce7-f3b2b6fedb71', metadata={'source': 'data/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer'),
 '716812c2-13e2-4a0a-8ae2-07a7300475b9': Document(id='716812c2-13e2-4a0a-8ae2-07a7300475b9', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.\n예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.\n연관키워

### **FAISS 벡터 저장소 생성 (from_texts)**

`from_texts` 를 사용한 Vector Store 생성(텍스트를 직접 넣으면서 DB 생성)

**매개변수**

- `texts` (List[str]): 벡터 저장소에 추가할 텍스트 리스트
- `embedding` (Embeddings): 사용할 임베딩 함수
- `metadatas` (Optional[List[dict]]): 메타데이터 리스트. 기본값은 None
- `ids` (Optional[List[str]]): 문서 ID 리스트. 기본값은 None
- `**kwargs`: 추가 키워드 인자

**동작 방식**

1. 제공된 임베딩 함수를 사용하여 텍스트를 임베딩
2. 임베딩된 벡터와 함께 `__from` 메서드를 호출하여 FAISS 인스턴스를 생성

In [45]:
# 문자열 리스트로 생성
db2 = FAISS.from_texts(
    ["안녕하세요.", "집 가고 싶어요"],
    embedding=OpenAIEmbeddings(),
    metadatas=[{"source": "텍스트문서"}, {"source": "텍스트문서"}],
    ids=["doc1", "doc2"],
)

# 저장된 내용
db2.docstore._dict

{'doc1': Document(id='doc1', metadata={'source': '텍스트문서'}, page_content='안녕하세요.'),
 'doc2': Document(id='doc2', metadata={'source': '텍스트문서'}, page_content='집 가고 싶어요')}

### **유사도 검색 (Similarity Search)**

`similarity_search` 메서드는 주어진 쿼리와 가장 유사한 문서들을 검색하는 기능을 제공

**매개변수**

- `query` (str): 유사한 문서를 찾기 위한 검색 쿼리 텍스트
- `k` (int): 반환할 문서 수. 기본값은 4
- `filter` (Optional[Union[Callable, Dict[str, Any]]]): 메타데이터 필터링 함수 또는 딕셔너리. 기본값은 None
- `fetch_k` (int): 필터링 전에 가져올 문서 수. 기본값은 20
- `**kwargs`: 추가 키워드 인자

**반환값**

- `List[Document]`: 쿼리와 가장 유사한 문서 리스트

**동작 방식**

1. `similarity_search_with_score` 메서드를 내부적으로 호출하여 유사도 점수와 함께 문서를 검색합니다.
2. 검색 결과에서 점수를 제외하고 문서만 추출하여 반환

**주요 특징**

- `filter` 매개변수를 사용하여 메타데이터 기반의 필터링이 가능
- `fetch_k`를 통해 필터링 전 검색할 문서 수를 조절할 수 있어, 필터링 후 원하는 수의 문서를 확보할 수 있음

**사용 시 고려사항**

- 검색 성능은 사용된 임베딩 모델의 품질에 크게 의존
- 대규모 데이터셋에서는 `k`와 `fetch_k` 값을 적절히 조정하여 검색 속도와 정확도의 균형을 맞추는 것이 중요
- 복잡한 필터링이 필요한 경우, `filter` 매개변수에 커스텀 함수를 전달하여 세밀한 제어가 가능

**최적화 팁**

- 자주 사용되는 쿼리에 대해서는 결과를 캐싱하여 반복적인 검색 속도를 향상시킬 수 있음
- `fetch_k`를 너무 크게 설정하면 검색 속도가 느려질 수 있으므로, 적절한 값을 실험적으로 찾는 것이 좋음

In [46]:
# 유사도 검색(k = 4)
db.similarity_search("TF IDF 에 대하여 알려줘")

[Document(id='93d64887-f21e-4169-a15f-b93e2f4b07d2', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='f23a3f29-fec4-4abb-99e6-f577f4ef2fda', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직된 데이터입니다. 이는 데이터베이

In [ ]:
# k 값 지정
db.similarity_search("TF IDF 에 대하여 알려줘", k=2)


🔍 검색 결과:
정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.
예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.
연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝

Deep Learning

정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.
예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.
연관키워드: 인공신경망, 머신러닝, 데이터 분석

Schema

정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.
예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.
연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리

DataFrame {'source': 'data/nlp-keywords.txt'}


In [ ]:
# filter 사용 soruce : 검색할 문서
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "data/nlp-keywords.txt"}, k=2
)

## **저장 및 로드**

### **로컬 저장 (Save Local)**

`save_local` 메서드는 FAISS 인덱스, 문서 저장소, 그리고 인덱스-문서 ID 매핑을 로컬 디스크에 저장 기능 제공

**매개변수**

- `folder_path` (str): 저장할 폴더 경로
- `index_name` (str): 저장할 인덱스 파일 이름 (기본값: "index")

**동작 방식**

1. 지정된 폴더 경로를 생성합니다 (이미 존재하는 경우 무시).
2. FAISS 인덱스를 별도의 파일로 저장
3. 문서 저장소와 인덱스-문서 ID 매핑을 pickle 형식으로 저장

**사용 시 고려사항**

- 저장 경로에 대한 쓰기 권한이 필요
- 대용량 데이터의 경우 저장 공간과 시간이 상당히 소요될 수 있음
- pickle 사용으로 인한 보안 위험을 고려해야 

In [47]:
# 로컬 Disk 에 저장
db.save_local(folder_path="./db/faiss_db", index_name="faiss_index")

### 로컬에서 불러오기 (Load Local)

`load_local` 클래스 메서드는 로컬 디스크에 저장된 FAISS 인덱스, 문서 저장소, 그리고 인덱스-문서 ID 매핑을 불러오는 기능을 제공

**매개변수**

- `folder_path` (str): 불러올 파일들이 저장된 폴더 경로
- `embeddings` (Embeddings): 쿼리 생성에 사용할 임베딩 객체
- `index_name` (str): 불러올 인덱스 파일 이름 (기본값: "index")
- `allow_dangerous_deserialization` (bool): pickle 파일 역직렬화 허용 여부 (기본값: False)

**반환값**

- `FAISS`: 로드된 FAISS 객체

**동작 방식**

1. 역직렬화의 위험성을 확인하고 사용자의 명시적 허가를 요구
2. FAISS 인덱스를 별도로 불러옵니다.
3. pickle을 사용하여 문서 저장소와 인덱스-문서 ID 매핑을 불러옴
4. 불러온 데이터로 FAISS 객체를 생성하여 반환

In [51]:
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="./db/faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

# 로드된 데이터를 확인
loaded_db.index_to_docstore_id

{0: '7f617280-ecb0-472d-9ce7-f3b2b6fedb71',
 1: '716812c2-13e2-4a0a-8ae2-07a7300475b9',
 2: 'e12c13b4-b17b-48d1-ad52-2be1797770d1',
 3: 'e4ee5a0e-fb4c-411e-b597-1b2c732a3db9',
 4: '5633d481-7eb9-4268-bd78-3353089125e4',
 5: 'f23a3f29-fec4-4abb-99e6-f577f4ef2fda',
 6: '93d64887-f21e-4169-a15f-b93e2f4b07d2',
 7: '1fc5573e-e88a-4d24-861a-1ba39f80ab8b',
 8: '295b1d8e-edb7-48a1-99d4-c95c469eebff',
 9: 'aa1465c0-0a2f-4654-9aff-8e04e2f80976',
 10: 'a5d31f54-f0be-482a-b1c6-32e98f239270'}

### **FAISS 객체 병합 (Merge From)**

`merge_from` 메서드는 현재 FAISS 객체에 다른 FAISS 객체를 병합하는 기능을 제공

**매개변수**

- `target` (FAISS): 현재 객체에 병합할 대상 FAISS 객체

**동작 방식**

1. 문서 저장소의 병합 가능 여부를 확인
2. 기존 인덱스의 길이를 기준으로 새로운 문서들의 인덱스를 설정
3. FAISS 인덱스를 병합
4. 대상 FAISS 객체의 문서와 ID 정보를 추출
5. 추출한 정보를 현재 문서 저장소와 인덱스-문서 ID 매핑에 추가

**주요 특징**

- 두 FAISS 객체의 인덱스, 문서 저장소, 인덱스-문서 ID 매핑을 모두 병합
- 인덱스 번호의 연속성을 유지하면서 병합
- 문서 저장소의 병합 가능 여부를 사전에 확인


**주의사항**

- 병합 대상 FAISS 객체와 현재 객체의 구조가 호환되어야 
- 중복 ID 처리에 주의해야  현재 구현에서는 중복 검사를 하지 않음
- 병합 과정에서 예외가 발생하면 부분적으로 병합된 상태가 될 수 있음

In [52]:
# 저장된 데이터를 로드
db = FAISS.load_local(
    folder_path="/db/faiss_db",
    index_name="faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,
)

# db 의 데이터 확인
db.index_to_docstore_id

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\bld\faiss-split_1723208774635\work\faiss\impl\io.cpp:68: Error: 'f' failed: could not open \db\faiss_db\faiss_index.faiss for reading: No such file or directory

In [53]:
# 새로운 FAISS 벡터 저장소 생성
db2 = FAISS.from_documents(documents=split_doc2, embedding=OpenAIEmbeddings())

# db2 의 데이터 확인
db2.index_to_docstore_id

{0: '342e42a9-770b-4673-b089-a7352c7626d5',
 1: '1de9afed-0961-451f-8b80-e96cda02e614',
 2: 'faf50341-4a5b-4f53-b6a6-418173848f2d',
 3: 'e26e8a2b-fedf-45c0-809c-2aeec0bb6966',
 4: '7d888492-3024-48b2-86f9-f4a85502c826',
 5: '806fc8ce-46eb-4797-87fb-c478071afc2d'}

In [56]:
# db + db2 를 병합
db.merge_from(db2)

In [57]:
# 병합된 데이터 확인
db.index_to_docstore_id

{0: '7f617280-ecb0-472d-9ce7-f3b2b6fedb71',
 1: '716812c2-13e2-4a0a-8ae2-07a7300475b9',
 2: 'e12c13b4-b17b-48d1-ad52-2be1797770d1',
 3: 'e4ee5a0e-fb4c-411e-b597-1b2c732a3db9',
 4: '5633d481-7eb9-4268-bd78-3353089125e4',
 5: 'f23a3f29-fec4-4abb-99e6-f577f4ef2fda',
 6: '93d64887-f21e-4169-a15f-b93e2f4b07d2',
 7: '1fc5573e-e88a-4d24-861a-1ba39f80ab8b',
 8: '295b1d8e-edb7-48a1-99d4-c95c469eebff',
 9: 'aa1465c0-0a2f-4654-9aff-8e04e2f80976',
 10: 'a5d31f54-f0be-482a-b1c6-32e98f239270',
 11: '342e42a9-770b-4673-b089-a7352c7626d5',
 12: '1de9afed-0961-451f-8b80-e96cda02e614',
 13: 'faf50341-4a5b-4f53-b6a6-418173848f2d',
 14: 'e26e8a2b-fedf-45c0-809c-2aeec0bb6966',
 15: '7d888492-3024-48b2-86f9-f4a85502c826',
 16: '806fc8ce-46eb-4797-87fb-c478071afc2d'}

### **검색기로 변환 (as_retriever)**

`as_retriever` 메서드는 현재 벡터 저장소를 기반으로 `VectorStoreRetriever` 객체를 생성하는 기능을 제공합니다.

### **매개변수**
- `**kwargs`: 검색 함수에 전달할 키워드 인자
  - `search_type` (Optional[str]): 검색 유형 (`"similarity"`, `"mmr"`, `"similarity_score_threshold"`)
  - `search_kwargs` (Optional[Dict]): 검색 함수에 전달할 추가 키워드 인자

### **반환값**
- `VectorStoreRetriever`: 벡터 저장소 기반의 검색기 객체

### **주요 기능**
1. **다양한 검색 유형 지원**
   - `"similarity"`: 유사도 기반 검색 (기본값)
   - `"mmr"`: Maximal Marginal Relevance 검색
   - `"similarity_score_threshold"`: 임계값 기반 유사도 검색

2. **검색 매개변수 커스터마이징**
   - `k`: 반환할 문서 수
   - `score_threshold`: 유사도 점수 임계값
   - `fetch_k`: MMR 알고리즘에 전달할 문서 수
   - `lambda_mult`: MMR 다양성 조절 파라미터
   - `filter`: 문서 메타데이터 기반 필터링

### **사용 시 고려사항**
- 검색 유형과 매개변수를 적절히 선택해 검색 결과의 품질과 다양성을 조절할 수 있음
- 대규모 데이터셋에서는 `fetch_k`와 `k` 값을 조절해 성능과 정확도의 균형을 맞출 수 있음
- 필터링 기능을 활용해 특정 조건에 맞는 문서만 검색할 수 있음

### **최적화 팁**
- MMR 검색 시 `fetch_k`를 높이고 `lambda_mult`를 조절해 다양성과 관련성의 균형을 맞출 수 있음
- 임계값 기반 검색을 사용해 높은 관련성을 가진 문서만 반환할 수 있음

### **주의사항**
- 부적절한 매개변수 설정은 검색 성능이나 결과의 품질에 영향을 줄 수 있음
- 대규모 데이터셋에서 높은 `k` 값 설정은 검색 시간을 증가시킬 수 있음

In [ ]:
# 새로운 FAISS 벡터 저장소 생성
db = FAISS.from_documents(
    documents=split_doc1 + split_doc2, embedding=OpenAIEmbeddings()
)

# 검색기로 변환(기본으로 4개 반환)
retriever = db.as_retriever()

# 검색 수행
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='bed9442f-a0ae-4daf-855a-5b4ddc193cbe', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(id='bb5f67b5-05f5-48e0-993d-dcb383d0cb78', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있

다양성이 높은 더 많은 문서 검색

- `k`: 반환할 문서 수 (기본값: 4)
- `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
- `lambda_mult`: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5)

In [59]:
# MMR 검색 수행
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25, "fetch_k": 10}
)
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='bed9442f-a0ae-4daf-855a-5b4ddc193cbe', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source'),
 Document(id='d1849746-4acb-4892-bb09-fb5717dc493d', metadata={'source': 'data/nlp-keywords.txt'}, page_content='GPT (Generative Pretrained Transformer)\n\n정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.\n예시: 사용자가 제공한 질문에 대해 자세한 답변을 생